In [ ]:
import numpy as np
from influxdb_client import InfluxDBClient, Point, Dialect
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
import datetime
import matplotlib.pyplot as plt

print("imports ok")


In [ ]:
# initialization
url_cita = "https://us-central1-1.gcp.cloud2.influxdata.com/"
token_cita = "Z-v8CQVLgaaqqS-cjDg-lKgYHR5tN3J2rclZxeNzbED9gDHVo1pYqnGdf25drjohEqM7JiAm_Q7ItW6CX0m9Gg=="
org_cita = "a769c173be65f818"

client = InfluxDBClient(url=url_cita, token=token_cita, org=org_cita)
query_api = client.query_api()

In [ ]:
#Set mocap variables
timeRange = "-30d"
expName= "demo_batch5_w5_w19_w20_w22"
measureList = ["x_pos", "y_pos", "z_pos"]

In [ ]:
#Get markers id     #MANUALLY SET TIME RANGE

queryTxtInit = 'from(bucket:"SensorBox") ' \
            '|> range(start: 2021-12-08T07:40:18.000Z, stop: 2021-12-17T08:00:00.000Z) ' \
            '|> filter(fn: (r) => r["_measurement"] == "%s") ' \
            '|> keyValues(keyColumns: ["markerID"])' % (expName)

print(queryTxtInit)

df_init = query_api.query_data_frame(queryTxtInit)
df_init = df_init[["markerID"]]
df_init = df_init.drop_duplicates(subset=None, keep="first", inplace=False)
df_init.sort_values(by=['markerID'])
#df_init.to_csv("markersID.csv")
print(df_init)

markerID_list = df_init['markerID'].tolist()
print("we have", len(markerID_list), "markers")
print(markerID_list)

In [ ]:
#GET MARKER DATA
allData = pd.DataFrame()

for i in range(len(markerID_list)):
#for i in range(10):
    #create dataFrame per marker
    markerDF = pd.DataFrame()

#     columns = [(str(markerID_list[i]),"x_pos"),(str(markerID_list[i]),"y_pos"),(str(markerID_list[i]),"z_pos")]
#     markerDF = pd.MultiIndex.from_tuples(columns)
#     print(markerDF)

    for j in range(len(measureList)):
        identifier = str(markerID_list[i]) + " " + str(measureList[j])
        print(identifier)

        # generate query text
        queryTxt = 'from(bucket:"SensorBox") ' \
                    '|> range(start: 2021-12-08T07:40:18.000Z, stop: 2021-12-17T08:00:00.000Z) ' \
                    '|> filter(fn: (r) => r["_measurement"] == "%s") ' \
                    '|> filter(fn: (r) => r["markerID"] == "%s") ' \
                    '|> filter(fn: (r) => r["_field"] == "%s" )' \
                    '|> keep(columns: ["_value", "_time"])' % (expName , markerID_list[i], measureList[j])
        #print(queryTxt)
        #query influx
        df = query_api.query_data_frame(queryTxt)
        df = df[["_time", "_value"]]
        df.columns = ["_time", identifier]

        #get rid of seconds and nanoseconds and make time as idx
        df["_time"] = df['_time'].astype(str).str[:16]

        #don't do this yet because we have to open in excel
        #pd.to_datetime(df["_time"])
        df.set_index("_time", inplace= True)
        #print(df)

        #add 3 positions together
        markerDF[identifier] = df[identifier]

    #print(markerDF)
    allData = pd.concat([allData, markerDF], axis=1)

print(allData.info())



In [ ]:
allData.columns = allData.columns.str.split(' ', expand=True)
allData.to_csv("allMarkers_demo_batch5_w5_w19_w20_w22_FULLTIME.csv")

###PAUSE

In [ ]:
#Load clean data
cleanMarkers = pd.read_csv("WIP_batch5.csv",index_col=[0], header=[0,1])
print(cleanMarkers)

In [ ]:
#GET ID FROM GRASSHOPPER

thisExp = [5,19,20]
block_5 = ["0_50873","0_50883","0_50886","0_50899","0_50904","0_50906","0_50908","0_50957"]
block_19 = ["0_50977","0_50990","0_51028","0_51036","0_51053","0_51059","0_51060","0_51061","0_51071","0_51434","0_51436","0_52922"]
block_20 = ["0_51321","0_51335","0_52071","0_52900","0_52916","0_52920","0_52923","0_52930","0_52933","0_52936"]


In [ ]:
for item in thisExp:
    globals()['df_block%s' % item] = cleanMarkers[globals()['block_%s' % item]]
    #print(globals()['df_block%s' % item])
    globals()['df_block%s' % item].to_csv('./CleanCSV/df_block%s_batch5.csv' % item)


In [ ]:
#Convert to datetime

#get experiment start as time
df = pd.DataFrame()
fmt = '%d/%m/%Y %H:%M'
df['_time'] = cleanMarkers.index
#Turn _time into datetime
df["_time"] = pd.to_datetime(df["_time"], format= fmt)

print(df)
print(df.info())


In [ ]:

df_block1 = cleanMarkers[block_1]
df_block1.set_index("_time", inplace= True)

df_block3 = cleanMarkers[block_3]
df_block3.set_index("_time", inplace= True)

df_block6 = cleanMarkers[block_6]
df_block6.set_index("_time", inplace= True)

df_block9 = cleanMarkers[block_9]
df_block9.set_index("_time", inplace= True)


In [ ]:
#Get clean marker list
markerList_clean = []

for name in cleanMarkers.columns:
    markerList_clean.append(name[0])
markerList_clean = set(markerList_clean)
markerList_clean = sorted(markerList_clean)
print(markerList_clean)

import math
for name in markerList_clean:
    #create col name, used later to build df
    colName = name + "_dist"

    #get point 0
    point0_x = cleanMarkers[(name, "x_pos")].iloc[0]
    point0_y = cleanMarkers[(name, "y_pos")].iloc[0]
    point0_z = cleanMarkers[(name, "z_pos")].iloc[0]
    #print("point 0", point0_x, point0_y, point0_z)

    distances = []

    for x in cleanMarkers.index:
        pt_x = cleanMarkers[(name, "x_pos")].loc[x]
        pt_y = cleanMarkers[(name, "y_pos")].loc[x]
        pt_z = cleanMarkers[(name, "z_pos")].loc[x]
        #print("p", pt_x, pt_y, pt_z)
        dist= math.sqrt((pt_x - point0_x)**2 + (pt_y - point0_y)**2 + (pt_z - point0_z)**2)
        distances.append(dist)

    df[colName] = distances

print(df)

(optional) experimental probes & other sensors

In [ ]:
#Get other Sensors
def getExpData(timeRange, expName, save, measuresList):
    """
    :param timeRange: string in flux langauge
    :param expName: string _measurement
    :param save: boolean save file as csv or not
    :return: clean dataframe with normalized curing time and _measurement as column name
    """

    for i in range(len(measuresList)):
        # Query InfluxDB

        queryTxt = 'from(bucket:"SensorBox") ' \
                   '|> range(start: %s) ' \
                   '|> filter(fn: (r) => r["_measurement"] == "%s") ' \
                   '|> filter(fn: (r) => r["_field"] == "%s" )' \
                   '|> keep(columns: ["_value", "_time"])' % (timeRange, expName , measuresList[i])

        print(queryTxt)

        df = query_api.query_data_frame(queryTxt)

        #clean data frame
        df.drop("result",inplace=True, axis=1)
        df.drop("table",inplace=True, axis=1)

        #get rid of seconds and nanoseconds
        df["_time"] = df['_time'].astype(str).str[:16]
        fmt = '%Y-%m-%d %H:%M'

        #set Time as index
        df["_time"] = pd.to_datetime(df["_time"])
        df.set_index("_time", inplace= True)

        #rebuild & rename dataframe
        df = df[["_value"]]

        valueTag = '%s-%s' % (expName, measuresList[i])

        df.columns = [valueTag]
        print(df.info())

        if i == 0:
            df_allMeasurement = df
        else:
            df_allMeasurement[valueTag] = df[valueTag]


    if save:
        # Save dataframe
        df_allMeasurement.to_csv("%s.csv" % (expName))
        print(df_allMeasurement.info())


    return df_allMeasurement

measuresList = ["Mat_weight", "Mat_moisture01", "Mat_moisture02"]

df_exp16_2= getExpData("-30d", "Exp_16.2", True, measuresList)


In [ ]:
#Create curing time

#expStart = datetime.datetime.strptime(df.iloc[0]["_time"], fmt)
df_circle_all["t"] = df_circle_all.index
df_circle_all["t"] = df_circle_all["t"].astype("str")
print(df_circle_all.info())

#get experiment start as time
fmt = '%Y-%m-%d %H:%M:%S'
expStart = datetime.datetime.strptime(df_circle_all.iloc[0]["t"], fmt)
print(expStart)

#Create curing time column, for duration normalization
for lab,row in df_circle_all.iterrows():
    df_circle_all.loc[lab,["curing time"]]= int(round(((datetime.datetime.strptime(row["t"], fmt)) - expStart).total_seconds() / 60))




#Create weight loss percentage
weightStart = df_circle_all.iloc[0]["Exp_16.2-Mat_weight"]
print(weightStart)

#Create weight percentage, for normalization
for lab,row in df_circle_all.iterrows():
    df_circle_all.loc[lab,["weight_loss_%"]]= row["Exp_16.2-Mat_weight"] * 100 / weightStart

df_circle_all = df_circle_all.drop(columns=['t', "Exp_16.2-Mat_weight"])
print(df_circle_all)

In [ ]:
#PLOTTING

fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('curing time (s)')
ax1.set_ylabel('%', color=color)
ax1.plot(df_circle_all["curing time"], df_circle_all["weight_loss_%"], label = "weight loss")
ax1.plot(df_circle_all["curing time"], df_circle_all["Exp_16.2-Mat_moisture01"], label = "water loss")
ax1.tick_params(axis='y', labelcolor=color)
xMax = df_circle_all["curing time"].max()
# for i in range(int(xMax // 1440)):
#   ax1.vlines(i *1440, 0, 100, label = "day %s" % (i+1), color = "black")

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('mocap dist (m)', color=color)  # we already handled the x-label with ax1
variables = ["0_50248_dist", "0_50249_dist", "0_50269_dist", "0_50270_dist", "0_50271_dist", "0_50272_dist", "0_50279_dist", "0_50281_dist","0_50304_dist"]
for i in range(len(variables)):
    ax2.plot(df_circle_all["curing time"], df_circle_all[variables[i]], label = "marker%s" % i)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()# otherwise the right y-label is slightly clipped
fig.set_figheight(10)
fig.set_figwidth(20)

ax1.legend(bbox_to_anchor=(1.05, 0.5), loc='upper left')
ax2.legend(bbox_to_anchor=(1.05, 0.75), loc='upper left')


plt.title('Cyclinder drying')
plt.show()